In [1]:
#hide
#default_exp sync_scala
#default_cls_lvl 3
from nbdev.showdoc import show_doc

In [2]:
#export
from nbdev.imports import *
# from nbdev.export import *
from fastcore.script import *
import nbformat
from nbformat.sign import NotebookNotary

In [3]:
#export
import sys
import os

SCRIPT_DIR = os.path.dirname(os.getcwd()) + '/chisel_nbdev'
sys.path.append(os.path.normpath(SCRIPT_DIR))
from export_scala import *

# Synchronize and diff

> The functions that propagates small changes in the library back to notebooks



The library is primarily developed in notebooks so any big changes should be made there. But sometimes, it's easier to fix small bugs or typos in the modules directly. `nbdev_update_lib` is the function that will propagate those changes back to the corresponding notebooks. Note that you can't create new cells with that functionality, so your corrections should remain limited.

## Finding the way back to notebooks

We need to get the name of the object we are looking for, and then we'll try to find it in our index file.

In [4]:
#export
def _get_property_name(p):
    "Get the name of property `p`"
    if hasattr(p, 'fget'):
        return p.fget.func.__qualname__ if hasattr(p.fget, 'func') else p.fget.__qualname__
    else: return next(iter(re.findall(r'\'(.*)\'', str(p)))).split('.')[-1]

def get_name(obj):
    "Get the name of `obj`"
    if hasattr(obj, '__name__'):       return obj.__name__
    elif getattr(obj, '_name', False): return obj._name
    elif hasattr(obj,'__origin__'):    return str(obj.__origin__).split('.')[-1] #for types
    elif type(obj)==property:          return _get_property_name(obj)
    else:                              return str(obj).split('.')[-1]

In [5]:
from nbdev.export import DocsTestClass

In [6]:
test_eq(get_name(in_ipython), 'in_ipython')
test_eq(get_name(DocsTestClass.test), 'test')

In [7]:
#export
def qual_name(obj):
    "Get the qualified name of `obj`"
    if hasattr(obj,'__qualname__'): return obj.__qualname__
    if inspect.ismethod(obj):       return f"{get_name(obj.__self__)}.{get_name(fn)}"
    return get_name(obj)

In [8]:
test_eq(qual_name(DocsTestClass.test), 'DocsTestClass.test')

In [9]:
#export
def source_nb(func, is_name=None, return_all=False, mod=None):
    "Return the name of the notebook where `func` was defined"
    is_name = is_name or isinstance(func, str)
    if mod is None: mod = get_nbdev_module()
    index = mod.index
    name = func if is_name else qual_name(func)
#     print(f'source_nb : name = {name}')
    while len(name) > 0:
        if name in index: return (name,index[name]) if return_all else index[name]
        name = '.'.join(name.split('.')[:-1])

You can either pass an object or its name (by default `is_name` will look if `func` is a string or not to determine if it should be `True` or `False`, but you can override if there is some inconsistent behavior). 

If passed a method of a class, the function will return the notebook in which the largest part of the function name was defined in case there is a monkey-matching that defines `class.method` in a different notebook than `class`. If `return_all=True`, the function will return a tuple with the name by which the function was found and the notebook.

For properties defined using `property` or our own `add_props` helper, we approximate the name by looking at their getter functions, since we don't seem to have access to the property name itself. If everything fails (a getter cannot be found), we return the name of the object that contains the property. This suffices for `source_nb` to work.

In [10]:
test_eq(source_nb("test.Add"), 'test.ipynb')
test_eq(source_nb("Add"), 'ToImport.ipynb')
test_eq(source_nb("Operator"), 'test.ipynb')
test_eq(source_nb("MulDiv"), 'test.ipynb')
assert source_nb(int) is None

## Reading the library

If someone decides to change a module instead of the notebooks, the following functions help update the notebooks accordingly.

In [11]:
#export
_re_cell = re.compile(r'^// Cell|^// Internal Cell|^// Comes from\s+(\S+), cell')

In [12]:
#export
def _split(code):
    lines = code.split('\n')
    nbs_path = Config().path("nbs_path").relative_to(Config().config_file.parent)
    prefix = '' if nbs_path == Path('.') else f'{nbs_path}/'
    default_nb = re.search(f'File to edit: {prefix}(\\S+)\\s+', lines[0]).groups()[0]
    s,res = 1,[]
    while _re_cell.search(lines[s]) is None: s += 1
    e = s+1
    while e < len(lines):
        while e < len(lines) and _re_cell.search(lines[e]) is None: e += 1
        grps = _re_cell.search(lines[s]).groups()
        nb = grps[0] or default_nb
        content = lines[s+1:e]
        while len(content) > 1 and content[-1] == '': content = content[:-1]
        res.append((nb, '\n'.join(content)))
        s,e = e,e+1
    return res

In [13]:
#export
def _script2notebook(fname, dic, silent=False):
    "Put the content of `fname` back in the notebooks it came from."
    if os.environ.get('IN_TEST',0): return  # don't export if running tests
    fname = Path(fname)
    with open(fname, encoding='utf8') as f: code = f.read()
    splits = _split(code)
    rel_name = fname.absolute().resolve().relative_to(Config().path("lib_path"))
    key = str(rel_name.with_suffix(''))
    assert len(splits)==len(dic[key]), f'"{rel_name}" exported from notebooks should have {len(dic[key])} cells but has {len(splits)}.'
    assert all([c1[0]==c2[1]] for c1,c2 in zip(splits, dic[key]))
    splits = [(c2[0],c1[0],c1[1]) for c1,c2 in zip(splits, dic[key])]
#     print(f'fname: {fname}, rel_name: {rel_name}, splits: {splits}')
    # get index, dest_filename, code of each cell in the script
    nb_fnames = {Config().path("nbs_path")/s[1] for s in splits}
    for nb_fname in nb_fnames:
#         print(f'nb_fname: {nb_fname}')
        nb = read_nb(nb_fname)
        for i,f,c in splits:
#             c = _deal_loc_import(c, str(fname))
#             print(f'{f} : {c}')
            if f == nb_fname.name:
                flags = split_flags_and_code(nb['cells'][i], str)[0]
                nb['cells'][i]['source'] = flags + '\n' + c
#                 print(f"updated cell to: {nb['cells'][i]['source']}")
#                 nb['cells'][i]['source'] = flags + '\n' + c.replace('', '')
            NotebookNotary().sign(nb)
            nbformat.write(nb, str(nb_fname), version=4)

    if not silent: print(f"Converted {rel_name}.")

In [14]:
#hide
dic = scala_notebook2script(silent=True, to_dict=True)
# pass in the module names to script2notebook
# print(dic)
_script2notebook(Config().path("lib_path")/'test.sc', dic)
_script2notebook(Config().path("lib_path")/'NewScript.sc', dic)
_script2notebook(Config().path("lib_path")/'NewScript2.sc', dic)

resetting /Users/jasonvranek/code/notebooks/chisel-nbdev/chisel_nbdev/_nbdev.py
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/07_clean_scala.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/03_export_scala2html.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/00_export_scala.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/02_show_scaladoc.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/01_sync_scala.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/08_export_sbt.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/04_test_scala.ipynb
Fail: /Users/jasonvranek/code/notebooks/chisel

In [15]:
#export
@call_parse
def nbdev_update_lib(fname:Param("A python filename or glob to convert", str)=None,
                     silent:Param("Don't print results", bool_arg)=False):
    "Propagates any change in the modules matching `fname` to the notebooks that created them"
#     if fname.endswith('.ipynb'): raise ValueError("`nbdev_update_lib` operates on .py files.  If you wish to convert notebooks instead, see `nbdev_build_lib`.")
    if os.environ.get('IN_TEST',0): return
    dic = scala_notebook2script(silent=True, to_dict=True)
    exported = get_nbdev_module().modules
#     print('exported', exported)

    if fname is None:
        files = [f for f in Config().path("lib_path").glob('**/*.sc') if str(f.relative_to(Config().path("lib_path"))) in exported]
#         print(files)
    else: files = glob.glob(fname)
    [ _script2notebook(f, dic, silent=silent) for f in files]

If `fname` is not specified, this will convert all modules and submodules in the `lib_folder` defined in `setting.ini`. Otherwise `fname` can be a single filename or a glob expression.

`silent` makes the command not print any statement. 

In [16]:
#hide
nbdev_update_lib()

resetting /Users/jasonvranek/code/notebooks/chisel-nbdev/chisel_nbdev/_nbdev.py
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/07_clean_scala.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/03_export_scala2html.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/00_export_scala.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/02_show_scaladoc.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/01_sync_scala.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/08_export_sbt.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/04_test_scala.ipynb
Fail: /Users/jasonvranek/code/notebooks/chisel

## Diff & trust notebooks

Before making a commit, you may want to check there is no diff between the exported library and the notebooks. You may also want to make this part of your CI, so that you don't accidentally merge a PR that introduces some changes between the two. This function is there to print this diff.

In [17]:
#export
import subprocess
from distutils.dir_util import copy_tree

In [18]:
#export
@call_parse
def nbdev_diff_nbs():
    "Prints the diff between an export of the library in notebooks and the actual modules"
    lib_folder = Config().path("lib_path")
    with tempfile.TemporaryDirectory() as d1, tempfile.TemporaryDirectory() as d2:
        copy_tree(Config().path("lib_path"), d1)
        scala_notebook2script(silent=True)
        copy_tree(Config().path("lib_path"), d2)
        shutil.rmtree(Config().path("lib_path"))
        shutil.copytree(d1, str(Config().path("lib_path")))
        for d in [d1, d2]:
            if (Path(d)/'__pycache__').exists(): shutil.rmtree(Path(d)/'__pycache__')
        res = subprocess.run(['diff', '-ru', d1, d2], stdout=subprocess.PIPE)
        print(res.stdout.decode('utf-8'))

If you receive an output, you'll need to either run `notebook2script()` or `nbdev_update_lib()` to fix the difference.

In [19]:
nbdev_diff_nbs()

resetting /Users/jasonvranek/code/notebooks/chisel-nbdev/chisel_nbdev/_nbdev.py
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/07_clean_scala.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/03_export_scala2html.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/00_export_scala.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/02_show_scaladoc.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/01_sync_scala.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/08_export_sbt.ipynb
Fail: Not creating mod file for non-scala notebook: /Users/jasonvranek/code/notebooks/chisel-nbdev/nbs/04_test_scala.ipynb
Fail: /Users/jasonvranek/code/notebooks/chisel

In [20]:
#export
@call_parse
def nbdev_trust_nbs(fname:Param("A notebook name or glob to convert", str)=None,
                    force_all:Param("Trust even notebooks that haven't changed", bool)=False):
    "Trust notebooks matching `fname`"
    check_fname = Config().path("nbs_path")/".last_checked"
    last_checked = os.path.getmtime(check_fname) if check_fname.exists() else None
    files = Config().path("nbs_path").glob('**/*.ipynb') if fname is None else glob.glob(fname)
    for fn in files:
        if last_checked and not force_all:
            last_changed = os.path.getmtime(fn)
            if last_changed < last_checked: continue
        nb = read_nb(fn)
        if not NotebookNotary().check_signature(nb): NotebookNotary().sign(nb)
    check_fname.touch(exist_ok=True)

In [21]:
nbdev_trust_nbs()